In [ ]:
import GPyOpt

# Define the optimization domain
# domain = [
#     {'name': 'learning_rate', 'type': 'continuous', 'domain': (0.0001, 1.0)},
    
#     # Discrete variable (integer values from a list)
#     {'name': 'num_layers', 'type': 'discrete', 'domain': (1, 2, 3, 4)},
# ]

domain = [
    {'name': 'num_units_layer', 'type': 'continuous', 'domain': (-5.12, 5.12)} for _ in range(400)
]

import numpy as np

# Example of Bayesian Optimization setup
def objective_function(x):
    # Your model training and evaluation logic here
    # x will contain values in the order specified in the domain
    return np.sum(x**2)  # Return the metric to optimize (e.g., validation loss)

optimizer = GPyOpt.methods.BayesianOptimization(
    f=objective_function,
    domain=domain,
    acquisition_type='EI',  #TODO: shuffle acquisition function
    exact_feval=True,
    initial_design_numdata=50,
    initial_design_type='latin',
)

# Run optimization
optimizer.run_optimization(max_iter=500, verbosity=True)

num acquisition: 1, time elapsed: 0.56s
num acquisition: 2, time elapsed: 1.19s
num acquisition: 3, time elapsed: 1.86s
num acquisition: 4, time elapsed: 2.49s
num acquisition: 5, time elapsed: 3.07s
num acquisition: 6, time elapsed: 3.66s
num acquisition: 7, time elapsed: 4.26s
num acquisition: 8, time elapsed: 4.86s
num acquisition: 9, time elapsed: 5.44s
num acquisition: 10, time elapsed: 6.07s
num acquisition: 11, time elapsed: 6.74s
num acquisition: 12, time elapsed: 7.34s
num acquisition: 13, time elapsed: 7.93s
num acquisition: 14, time elapsed: 8.50s
num acquisition: 15, time elapsed: 9.09s
num acquisition: 16, time elapsed: 9.73s
num acquisition: 17, time elapsed: 10.32s
num acquisition: 18, time elapsed: 10.91s
num acquisition: 19, time elapsed: 11.52s
num acquisition: 20, time elapsed: 12.10s
num acquisition: 21, time elapsed: 12.74s
num acquisition: 22, time elapsed: 13.47s
num acquisition: 23, time elapsed: 14.02s
num acquisition: 24, time elapsed: 14.56s
num acquisition: 

In [5]:
optimizer.save_evaluations('evaluations.txt')

In [ ]:
sorted(optimizer.Y, reverse=True)

[array([3055.56886577]),
 array([3088.73514679]),
 array([3111.3347072]),
 array([3129.25554746]),
 array([3146.75391419]),
 array([3153.0456014]),
 array([3154.21772189]),
 array([3154.78591088]),
 array([3164.93115725]),
 array([3169.25596695]),
 array([3170.22273536]),
 array([3185.28128766]),
 array([3194.85673859]),
 array([3195.78102735]),
 array([3201.17098732]),
 array([3203.33595461]),
 array([3204.18510476]),
 array([3204.61683758]),
 array([3210.37772866]),
 array([3211.41080064]),
 array([3212.33251501]),
 array([3213.41052835]),
 array([3217.48349268]),
 array([3220.59154563]),
 array([3221.5543325]),
 array([3223.24813173]),
 array([3227.88740527]),
 array([3229.11076352]),
 array([3229.4582922]),
 array([3231.23598593]),
 array([3246.05206548]),
 array([3248.6685539]),
 array([3250.83505119]),
 array([3251.08891648]),
 array([3252.3904541]),
 array([3254.72635084]),
 array([3260.07876316]),
 array([3260.8320885]),
 array([3269.78009368]),
 array([3276.07044379]),
 array(

In [2]:
from GPyOpt.acquisitions.base import AcquisitionBase



In [3]:
%pip install PySOT

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from pySOT.auxiliary_problems import candidate_dycors #, RBFInterpolant, LatinHypercube, OptimizationProblem
from pySOT.optimization_problems import OptimizationProblem
from poap.controller import SerialController

from pySOT.strategy import DYCORSStrategy
from pySOT.experimental_design import LatinHypercube
from pySOT.surrogate import RBFInterpolant  


#TODO: fix parameters by changing bounds in problem

# ===== DEFINE YOUR CUSTOM FUNCTION =====
class CustomProblem(OptimizationProblem):
    def __init__(self, dim=30):
        self.dim = dim
        self.lb = -5.12 * np.ones(dim)  # Lower bounds for all dimensions
        self.ub = 5.12 * np.ones(dim)   # Upper bounds for all dimensions
        self.cont_var = np.arange(dim)  # All variables are continuous
        self.int_var = []  # No integer variables
        self.info = "Square sum"

    def eval(self, x):
        return np.sum(x**2)

# Calculate the minimal value of the Rastrigin function
# optimal_x = np.zeros(dim)  # The global minimum is at x = 0 for all dimensions
# minimal_value = CustomProblem(dim=dim).eval(optimal_x)
# minimal_value

# ===== INITIALIZE PROBLEM =====
dim = 5  # Dimension of your problem
problem = CustomProblem(dim=dim)

# ===== CONFIGURE DYCORS =====
max_evaluations = 10  # TOTAL FUNCTION EVALUATIONS (including initial samples)
initial_points = 2 * dim + 1  # Initial design points (Latin Hypercube)

strategy = DYCORSStrategy(
    max_evals=max_evaluations,
    opt_prob=problem,
    exp_design=LatinHypercube(dim=dim, num_pts=initial_points),
    surrogate=RBFInterpolant(dim=dim, lb=problem.lb, ub=problem.ub),
    num_cand= 5,  # Candidate points per iteration
)

# startegy.problem.lb = ***


# ===== RUN OPTIMIZATION =====
controller = SerialController(objective=problem.eval)
controller.strategy = strategy
result = controller.run()

In [14]:
controller.fevals[0].params

(array([-3.70052056,  3.03649675, -3.33069734,  2.09941285,  4.0542684 ]),)

In [13]:
controller.fevals[0].value
# controller.fevals[0].value

54.852336276315214

In [12]:
-3.70052056 ** 2 +  3.03649675 ** 2 + -3.33069734 **2 +  2.09941285 **2 +  4.0542684 ** 2

5.277541901078454

In [17]:
np.sum(controller.fevals[0].params[0] ** 2)

54.852336276315214

In [ ]:
strategy.X

array([[-3.70052056,  3.03649675, -3.33069734,  2.09941285,  4.0542684 ],
       [-4.6063736 , -1.05373904, -2.08274776,  0.80189841, -1.37668381],
       [-2.9854332 ,  3.90439355,  1.24875227,  4.91347727, -2.00697926],
       [ 4.478361  ,  4.8819204 ,  0.40641288, -3.80646671, -0.17901644],
       [ 3.72889653, -4.57039445,  3.46464685, -1.1227966 , -3.5986247 ],
       [ 1.21611915, -3.00548502,  2.10578918, -5.06341203,  1.64403678],
       [ 2.21797377,  1.52327843, -1.05154372,  2.91900333, -2.90700582],
       [ 2.96947894, -1.62885395, -4.33840394, -2.15246722,  1.21162218],
       [-0.13033983, -3.2877341 ,  2.38428494,  4.09557824,  2.63111118],
       [-2.14715521, -0.07377683,  4.86106096,  0.26783739,  4.56353161]])

In [28]:
strategy.fX

array([[54.85233628],
       [29.20518133],
       [53.88670723],
       [58.57527116],
       [61.00772445],
       [43.28723233],
       [25.31679225],
       [36.3938625 ],
       [40.20750582],
       [49.1431898 ]])

In [25]:
controller.strategy.fX

array([[54.85233628],
       [29.20518133],
       [53.88670723],
       [58.57527116],
       [61.00772445],
       [43.28723233],
       [25.31679225],
       [36.3938625 ],
       [40.20750582],
       [49.1431898 ]])